## Installation de GeneticSharp


In [2]:
#r "nuget: GeneticSharp"

Installed Packages GeneticSharp, 3.1.4

## Définition et création du modèle de portefeuille

Nous modélisons ici un portefeuille d'actifs. Chaque actif est muni d'un prix, d'un rendement attendu, et participe à une matrice de covariance globale prise en compte dans la fonction de risque.  

In [3]:
using System;
using System.Collections.Generic;
using System.Linq;

// Classe Portfolio pour représenter un portefeuille d'actifs financiers
public class Portfolio
{
    // Liste des noms d'actifs
    public List<string> Assets { get; set; }

    // Dictionnaire avec les prix des actifs
    public Dictionary<string, double> Prices { get; set; }

    // Dictionnaire contenant les rendements attendus des actifs
    public Dictionary<string, double> ExpectedReturns { get; set; }

    // Matrice de covariance (taille NxN pour N actifs). 
    // CovarianceMatrix[i,j] représente la covariance entre l'actif i et j.
    public double[,] CovarianceMatrix { get; set; }

    // Constructeur pour initialiser le portefeuille
    public Portfolio(List<string> assets,
                     Dictionary<string, double> prices,
                     Dictionary<string, double> expectedReturns,
                     double[,] covarianceMatrix)
    {
        Assets = assets;
        Prices = prices;
        ExpectedReturns = expectedReturns;
        CovarianceMatrix = covarianceMatrix;
    }

    // Calcul du retour total du portefeuille
    public double CalculateReturn(Dictionary<string, double> weights)
    {
        double portfolioReturn = 0;
        foreach (var asset in Assets)
        {
            portfolioReturn += weights[asset] * ExpectedReturns[asset];
        }
        return portfolioReturn;
    }

    // Calcul du risque (écart-type) du portefeuille en utilisant la matrice de covariance
    public double CalculateRisk(Dictionary<string, double> weights)
    {
        // On convertit le dictionnaire de poids en un vecteur aligné sur l'ordre de la liste Assets
        var weightVector = new double[Assets.Count];
        for(int i = 0; i < Assets.Count; i++)
        {
            weightVector[i] = weights[Assets[i]];
        }

        // Calcul de la variance via wᵀΣw
        double variance = 0.0;
        int n = Assets.Count;
        for(int i = 0; i < n; i++)
        {
            for(int j = 0; j < n; j++)
            {
                variance += weightVector[i] * weightVector[j] * CovarianceMatrix[i, j];
            }
        }

        // Le risque est la racine carrée de la variance
        return Math.Sqrt(variance);
    }
}

## Définition du chromosome pour l'algorithme génétique

Chaque chromosome représente une allocation de portefeuille. Chaque gène correspond au poids d'un actif, et la somme des poids est toujours normalisée à 1.

In [4]:
using GeneticSharp;
using System.Linq;

// Classe pour représenter un chromosome de portefeuille
public class PortfolioChromosome : ChromosomeBase
{
    private List<string> _assets;

    // Constructeur : on crée un chromosome avec un gène par actif
    public PortfolioChromosome(List<string> assets) : base(assets.Count)
    {
        _assets = assets;
        for (int i = 0; i < assets.Count; i++)
        {
            ReplaceGene(i, new Gene(RandomizationProvider.Current.GetDouble(0, 1)));
        }
    }

    // Génère un gène avec une valeur aléatoire entre 0 et 1
    public override Gene GenerateGene(int geneIndex)
    {
        return new Gene(RandomizationProvider.Current.GetDouble(0, 1));
    }

    // Crée un nouveau chromosome
    public override IChromosome CreateNew()
    {
        return new PortfolioChromosome(_assets);
    }

    // Récupère la distribution des poids, normalisés pour que la somme = 1
    public Dictionary<string, double> GetWeights()
    {
        var weights = new Dictionary<string, double>();
        for (int i = 0; i < Length; i++)
        {
            weights.Add(_assets[i], (double)GetGene(i).Value);
        }
        NormalizeWeights(weights);
        return weights;
    }

    // Normalise les poids pour que leur somme soit égale à 1
    private void NormalizeWeights(Dictionary<string, double> weights)
    {
        double sum = weights.Values.Sum();
        var keys = weights.Keys.ToList();
        foreach (var key in keys)
        {
            weights[key] /= sum;
        }
    }
}

## Implémentation du Solveur par Algorithme Génétique

L'implémentation du solveur par algorithme génétique se fait en plusieurs étapes, incluant la définition des classes nécessaires, la configuration de l'algorithme génétique et l'exécution de l'algorithme pour trouver la meilleure solution possible.

In [5]:
// Classe pour évaluer la fitness d'un portefeuille
public class PortfolioFitness : IFitness
{
    private readonly Portfolio _portfolio;
    private readonly double _alpha;

    // Constructeur: le paramètre alpha permet de moduler l'aversion au risque
    public PortfolioFitness(Portfolio portfolio, double alpha = 1.0)
    {
        _portfolio = portfolio;
        _alpha = alpha;
    }

    // Fonction d'évaluation de la fitness
    public double Evaluate(IChromosome chromosome)
    {
        var portfolioChromosome = chromosome as PortfolioChromosome;
        var weights = portfolioChromosome.GetWeights();

        double portfolioReturn = _portfolio.CalculateReturn(weights);
        double risk = _portfolio.CalculateRisk(weights);

        // Objectif : maximiser le retour, minimiser le risque.
        // Fitness = retour - alpha*risk
        double fitness = portfolioReturn - _alpha * risk;
        return fitness;
    }
}

## Exemple d'utilisation et exécution de l'algorithme génétique

Dans cet exemple, nous générons cinq actifs différents avec des rendements attendus croissants.  
Une matrice de covariance simple leur est associée.  
Nous configurerons ensuite l'algorithme génétique avec une sélection par roulette, un crossover uniforme et une légère mutation.  
On ajoutera également un peu d'élitisme et un nombre de générations supérieur afin de réellement progresser dans la recherche d'une solution optimale.  

In [8]:
using GeneticSharp;

// Liste d'actifs
var assets = new List<string> { "Asset0", "Asset1", "Asset2", "Asset3", "Asset4" };

// Prix (exemple de dummy data)
var prices = new Dictionary<string, double>
{
    { "Asset0", 100 },
    { "Asset1", 200 },
    { "Asset2", 300 },
    { "Asset3", 400 },
    { "Asset4", 500 }
};

// Rendements attendus
var expectedReturns = new Dictionary<string, double>
{
    { "Asset0", 0.05 },
    { "Asset1", 0.10 },
    { "Asset2", 0.15 },
    { "Asset3", 0.20 },
    { "Asset4", 0.25 }
};

// Matrice de covariance (5x5) hypothétique
double[,] covarianceMatrix = new double[,]
{
    { 0.0100, 0.0012, 0.0018, 0.0021, 0.0025 },
    { 0.0012, 0.0200, 0.0022, 0.0026, 0.0028 },
    { 0.0018, 0.0022, 0.0300, 0.0031, 0.0033 },
    { 0.0021, 0.0026, 0.0031, 0.0400, 0.0043 },
    { 0.0025, 0.0028, 0.0033, 0.0043, 0.0500 }
};

// Instanciation du portefeuille
var portfolio = new Portfolio(assets, prices, expectedReturns, covarianceMatrix);

// Paramètre alpha = 1.0 (vous pouvez l'ajuster selon votre tolérance au risque)
var fitness = new PortfolioFitness(portfolio, alpha: 1.0);

// On crée un chromosome représentatif
var chromosome = new PortfolioChromosome(assets);

// Population initiale de 50 solutions, jusqu'à 100 solutions, sur la base du chromosome ci-dessus
var population = new Population(50, 100, chromosome);

// Configuration de l'algorithme génétique
var ga = new GeneticAlgorithm(
    population,
    fitness,
    new RouletteWheelSelection(),
    new UniformCrossover(),
    new UniformMutation()
);

// Nombre maximum de générations
ga.Termination = new GenerationNumberTermination(150);

// On peut configurer un peu d'élitisme et ajuster les probabilités
ga.Selection = new EliteSelection();
ga.MutationProbability = 0.05f;
ga.CrossoverProbability = 0.8f;

// Démarrage de l'algorithme
ga.Start();

// Récupération du meilleur chromosome
var bestChromosome = ga.BestChromosome as PortfolioChromosome;
var bestWeights = bestChromosome.GetWeights();

// Affichage des allocations
Console.WriteLine("=== Meilleure allocation d'actifs trouvée ===");
foreach (var asset in bestWeights.Keys)
{
    Console.WriteLine($"{asset,-10}: {bestWeights[asset]:P2}");
}

// Affichage du rendement et du risque
double bestReturn = portfolio.CalculateReturn(bestWeights);
double bestRisk = portfolio.CalculateRisk(bestWeights);
Console.WriteLine($"\nRendement attendu : {bestReturn:P2}");
Console.WriteLine($"Risque (écart-type): {bestRisk:P2}");
Console.WriteLine($"Fitness (return - alpha*risk): {bestReturn - bestRisk:0.0000}");

=== Meilleure allocation d'actifs trouvée ===
Asset0    : 0,17 %
Asset1    : 2,68 %
Asset2    : 28,39 %
Asset3    : 32,41 %
Asset4    : 36,36 %

Rendement attendu : 20,11 %
Risque (écart-type): 12,51 %
Fitness (return - alpha*risk): 0,0760


## Conclusion

Avec cet exemple plus élaboré, nous prenons mieux en compte la structure de corrélation/risque entre actifs grâce à la matrice de covariance, tout en modulant l’aversion au risque avec un paramètre alpha.  
L’algorithme génétique est configuré pour un plus grand nombre de générations, de l’élitisme, et des probabilités de croisement et de mutation afin d’accroître ses chances de trouver une allocation de portefeuille optimale.  

Pour aller encore plus loin, vous pourriez :  
• Expérimenter avec différentes méthodes de sélection (RouletteWheel, Tournament, Elite, etc.).  
• Ajuster la valeur de alpha pour augmenter ou diminuer la pénalisation du risque.  
• Introduire des contraintes supplémentaires (ex. poids minimum/maximum par actif).  
• Logger l’évolution de la fitness dans le temps pour observer la convergence de l’algorithme.  